## Gait Video Study 
### Identifying frames with HSRs in each video for each cohort and trial to establish break points and also evaluate the corresponding HSR labelling via the ground truth available. Further, downsample with smoothing to define fixed shape of the input tensor for models. 
#### Remember to preserve the original count of frames in a single stride (before down sampling via smoothing) for each stride to add as an additional artificial feature later to add information about speed of the subject to the model

In [108]:
import numpy as np
import cv2
import os
import glob
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
import scipy
from scipy import signal
from scipy.signal import lfilter, firwin, filtfilt, find_peaks, argrelextrema
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML

In [109]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data'
frame_path_merged = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\multi_view_merged_data\\'
#Folder to store the downsampled hip height normalized multi view merged files for models 
downsample_path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\downsampled_data\\'

#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

order = ['right hip', 'right knee', 'right ankle', 'left hip', 'left knee', 'left ankle', 'left toe 1', 'left toe 2', \
         'left heel', 'right toe 1', 'right toe 2', 'right heel']

In [110]:
#Saving the HSRframes.txt file to the hip_height_normalized\\ containing the final .csvs for analysis
# for cohort in cohorts:
#     for trial in trials:
#         merged_path = frame_path_merged+cohort+trial 
#         if (os.path.exists(merged_path)):
#             videos = os.listdir(merged_path)
# #             print (len(videos))
#         for video in videos:
#             print (glob.glob(path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'))
#             try:
#                 if (not os.path.exists(merged_path+'\\'+video+'\\HSRframes.txt')):
#                     HSR_frames_file = path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'+'\\HSRframes.txt'
#                     shutil.copy(HSR_frames_file, merged_path+'\\'+video+'\\hip_height_normalized\\') 
#                     print ('HSR for', video, 'copied')
#                 else:
#                     print ('HSR for', video, 'exists')
#             except Exception as e:
#                 print (e)

### Utility functions 

In [111]:
# Function to create a dataframe to hold all 12*3 features (right hip-x, right hip-y, ...) 
#as columns and all frames as rows for each video 
def compute_video_features(sorted_frames, labels):
    '''
    Input: sorted frames for the video and labels for the dataframe
    Returns: dataframe to hold all 12*3 features as columns and all frames as rows
    '''
    video_features = pd.DataFrame(columns = labels)
    for frame in sorted_frames:
        #Append the frame number also in the temp dataframe, since it the 
        #true HSR is given in frame number
        frame_no = int(frame.split('\\')[-1][:-4])
        frame_csv = pd.read_csv(frame, index_col = 0)
        #Appending the 36 features and frame number of each frame as 
        #a row for each video's dataframe 
        video_features.loc[len(video_features)] = np.append(frame_no, frame_csv[['x', 'y', 'z']].values.flatten())
    video_features = video_features.astype({'frame_number': 'int'})
    #Setting the frame number as the index 
    video_features.set_index('frame_number', inplace = True)
    #                 display(video_features)
    return video_features

In [112]:
#Function to plot the given feature across frames of a video and the corresponding ground truth and detected HSRs
def plot_true_HSR(signal, index, HSRindices, video, detectedHSR):
    fig= plt.figure(figsize = (8.5, 3))
    ax1 = fig.add_subplot(111)
    #Right heel height series 
    ax1.plot(index, signal, color = 'b', ls='solid', label = 'right heel height')
    #True HSRs
    ax1.plot(index[HSRindices], signal[HSRindices], 'r*', ms = 8, alpha = 1, label = 'true HSR')
    #Detected HSRs
    ax1.plot(index[detectedHSR], signal[detectedHSR], 'gd', ms = 6, alpha = 0.5, label = 'detected HSR')
    plt.xticks(index[0::40], fontsize = 8)
    plt.legend()
    plt.title('Detected and true HSRs using filtered right heel height')
    plt.show()
    plt.savefig('HSR_detection_figs_values\\'+ video+ '_filtered_right_heel_z.png', dpi = 300)
#     plt.close()

In [113]:
#Function to compute the detected HSRs from the right heel height series 
#Briefly, it fills missing values, filters, detects local minimas at a pre-assumed distance of 30 frames (1 second)
def compute_HSR(series_complete, trueHSR_indices_complete, video):
    '''
    Input: right heel height series, true HSR indices, video name
    Returns: Detected HSRs
    '''
    #First we use index aware linear interpolation to fill up the right heel height for the missing frames
    series_complete.interpolate(method='values', inplace = True)
    
    #Filtering the series using low pass FIR filter with cutoff of 0.3 i.e.   
    #0.3 = cutoff_in_Hz/Nyquist rate of the signal  where Nyquist rate of the signal = sample rate/2 = 30 frames/2 
    #hence, cutoff_in_Hz = 0.3*15 = 4.5 frames 
    #Further, length of the filter (number of coefficients, i.e. the filter order + 1) is 5
    a = signal.firwin(5, cutoff = 0.3, window = "hamming") 
    #Low pass filter with window length of 5 and cutoff frequency of 0.3/4.5 frames
    fir_series = signal.lfilter(a, [1.0], series_complete.values)
    
    #Identifying local minimum in right heel's height to define HSRs
    #Assuming that HSRs are atleast 30 frames apart (i.e. atleast a second apart)
    detectedHSR, _ = find_peaks(-fir_series, distance = 30) 
    
    #Plotting the right heel height (with the missing frames filled) and ground truth HSR 
    plot_true_HSR(fir_series, series_complete.index, trueHSR_indices_complete, video, detectedHSR)
    return detectedHSR

In [114]:
#Function to compute the error between the true and detected HSRs for a video
def compute_error_HSR(detectedHSR, trueHSR_indices):
    '''
    Input: A list of true and detected HSR indices for a video
    Returns: error list, absolute error list, mean, standard deviation error over a video (in frames) and 
    mean, standard deviation of absolute error over a video (in frames)
    To convert to seconds, divide by 30, since we have 30 frames per second.
    '''
    abs_error_indices = [np.argmin(abs(i-trueHSR_indices)) for i in detectedHSR]
    error = [i-trueHSR_indices[j] for i, j in zip(detectedHSR, abs_error_indices)]
    abs_error = list(map(abs, error))
    error_stats = [np.mean(error), np.std(error), np.mean(abs_error), np.std(abs_error)]
    return error, abs_error, error_stats

In [115]:
def rename(dataframe):
    dataframe['cohort'][dataframe.cohort=='\\ExtraHOA'] = 'HOA'
    dataframe['cohort'][dataframe.cohort=='\\HOA'] = 'HOA'
    dataframe['cohort'][dataframe.cohort=='\\MS'] = 'MS'
    dataframe['cohort'][dataframe.cohort=='\\PD'] = 'PD'
    dataframe['trial'][dataframe.trial=='\\beam_walking'] = 'BW'
    dataframe['trial'][dataframe.trial=='\\walking'] = 'W'

### main() function 

In [116]:
labels = ['frame_number'] + [o + '-'+ y for o in order for y in ['x', 'y', 'z']]
#Saves cohort, trial, video name, mean, std, abs mean, abs std for the error series, count of true HSRs
#, count of detected HSRs, the actual error series for each video and the absolute error series for each video
cols = ['cohort', 'trial', 'video', 'heel_mean_error', 'heel_std_error', 'heel_abs_mean_error',\
        'heel_abs_std_error', 'count_trueHSR', 'count_detectedHSR', 'error_list', 'abs_error_list']
dataframe_error_stats = pd.DataFrame(columns = cols)

In [ ]:
for cohort in cohorts:
    for trial in trials:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos:
            print (video)
            #Reading the ground truth HSR frames numbers
            trueHSR = open(merged_path+'\\'+video+'\\hip_height_normalized\\'+'\\HSRframes.txt').read()
            #Making a list containing ground truth HSRs out of the read file
            trueHSR_list = [int(a) for a in trueHSR.split(',')]
            count_trueHSR = len(trueHSR_list) #Count of true HSRs
            
            frames = glob.glob(merged_path+'\\'+video+'\\hip_height_normalized\\*.csv')
            #First, we need to sort the frames since we need frames to appear 
            #in order to detect HSRs
            sorted_frames = sorted(frames,  key=lambda name: int(name.split('\\')[-1][:-4]))
            #Dataframe to hold all 12*3 features (right hip-x, right hip-y, ...) 
            #as columns and all frames as rows for each video 
            video_features = compute_video_features(sorted_frames, labels)
            #If some frames are missing, appending those rows with NaN values 
            video_features_complete = video_features.reindex(range(video_features.index[-1]))
            
            #Indices for the ground truth HSR in the video_features_complete dataframe 
            #(containing NaN valued placeholders for the missing frames)
            trueHSR_indices_complete = [i for i, val in enumerate(video_features_complete.index) if val in trueHSR_list] 

            #Right heel height with missing frames encoded as NaNs
            series_complete = video_features_complete['right heel-z']
            #Computing the detected HSRs
            detectedHSR = compute_HSR(series_complete, trueHSR_indices_complete, video)
            #Saving the detected HSRs to a new .txt file
            detectedHSR_file = open('HSR_detection_figs_values\\'+ video+ '_detectedHSRs.txt', 'w') 
            detectedHSR_file.writelines(', '.join([str(x) for x in detectedHSR])) 
            detectedHSR_file.close()
            #Computing the error, absolute error and error stats for the detected and true HSR of the video
            HSR_error, HSR_abs_error, heel_error_stats = compute_error_HSR(detectedHSR, trueHSR_indices)
            #Saving the errors and corresponding stats to a dataframe for error analysis 
            dataframe_error_stats.loc[len(dataframe_error_stats)] = [cohort]+[trial]+[video]+ heel_error_stats + \
            [count_trueHSR] + [len(detectedHSR)] + [HSR_error] + [HSR_abs_error]
#Saving the csv with errors for error analysis later 
dataframe_error_stats.to_csv('HSR_detection_figs_values\\dataframe_error_stats_HSR_identification.csv')

GVS_212_T_T1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

GVS_212_T_T2


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

GVS_213_T_T1


...

GVS_314_T_T1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

GVS_314_T_T2


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

GVS_318_T_T1


In [10]:
dataframe_error_stats.mean() #cutoff 0.3

heel_mean_error         3.435317
heel_std_error         11.695836
heel_abs_mean_error    10.051407
heel_abs_std_error      7.956026
toe1_mean_error         3.950464
toe1_std_error         11.176331
toe1_abs_mean_error     9.686243
toe1_abs_std_error      7.903263
toe2_mean_error         3.845834
toe2_std_error         10.966280
toe2_abs_mean_error     9.540808
toe2_abs_std_error      7.739137
dtype: float64

### HSR identification error analysis 
Refer https://pubmed.ncbi.nlm.nih.gov/18657816/ for evaluation strategies 
1. Qualitative plot 
2. Distribution/histogram of error (use mean, not absolute mean) estimates 
3. Mean, SD for overall errors and analysis on HOA/MS/PD subgroups 
4. t-test for significant difference wrt to a reference error (collapse across each subject, so that we do not have repeated measures across same subject)
5. Linear mixed model trial/cohort to check differences between cohorts and trials 

In [22]:
#Do we need the missing frame information 
#We might need to annotate the ground truth HSR frames in the plot 
#https://stackoverflow.com/questions/14432557/matplotlib-scatter-plot-with-different-text-at-each-data-point
#For any algorithm to find the HSR's, we might need to fill missing values in the series we are using to detect HSR
#The ground truth HSR markers are not shown when the values are NAN- solve?



In [ ]:
[int(i) for i in str(HSR_error)[1:-1].split(',')]

In [16]:
plt.figure()
dataframe_error_stats['heel_mean_error'].hist()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#Add the SD also in cohort wise analysis 
HOA-7.27161818
MS-11.73048726
PD-12.39653977

### Downsample with smoothing to define fixed shape input tensor for models
The downsampled final .csvs are stored in C:\Users\purpl\Box\Gait Video Project\GaitVideoData\video\downsampled_data

In [11]:
## Run only once to create directories 
#To create all directories for saving the downsampled hip height normalized multi view merged data files 
# for cohort in cohorts:
#     for trial in trials:   
#         merged_path = frame_path_merged+cohort+trial 
#         if (os.path.exists(merged_path)):
#             videos = os.listdir(merged_path)
# #             print (len(videos))
#             for video in videos:         
#                 if not os.path.exists(downsample_path+cohort+trial+'\\'+video):
#                     os.makedirs(downsample_path+cohort+trial+'\\'+video)

In [ ]:
#Use mean with disjoint windows to downsample while smoothing 
#Make sure to preserve count of frames in a stride before smoothing to add as a feature 

